In [48]:
import pandas as pd
from sqlalchemy import create_engine, text

# Use the DATABASE_URL from db.py
DATABASE_URL = "postgresql://work@localhost/properties"

# Create the engine
engine = create_engine(DATABASE_URL)

# Query all properties from the database
query = "SELECT * FROM properties"

# Load the data into a DataFrame
with engine.connect() as conn:
    df = pd.read_sql_query(text(query), conn)

print(f"Number of records in df: {df.shape[0]}")

Number of records in df: 13898


In [49]:
# Convert lot_area from sqft to acres where lot_area_unit is 'sqft'
df.loc[df['lot_area_unit'] == 'sqft', 'lot_area_value'] = df.loc[df['lot_area_unit'] == 'sqft', 'lot_area_value'] / 43560
df.loc[df['lot_area_unit'] == 'sqft', 'lot_area_unit'] = 'acres'


In [50]:
# Export all properties to a CSV file
df.to_csv('/Users/work/Desktop/all_properties.csv', index=False)


In [51]:
from datetime import datetime

# Calculate the number of days since the property was sold
df['days_since_sold'] = (datetime.now() - pd.to_datetime(df['date_sold'])).dt.days

# Group by zip code and calculate the required metrics
zip_code_stats = df.groupby('zip_code').agg(
    properties_for_sale=('listing_status', lambda x: (x == 'FOR_SALE').sum()),
    properties_sold_90_days=('listing_status', lambda x: ((x == 'RECENTLY_SOLD') & (df.loc[x.index, 'days_since_sold'] <= 90)).sum()),
    properties_sold_30_days=('listing_status', lambda x: ((x == 'RECENTLY_SOLD') & (df.loc[x.index, 'days_since_sold'] <= 30)).sum()),
    average_price=('price', 'mean'),
    median_price=('price', 'median'),
    price_std=('price', 'std'),
    average_acreage=('lot_area_value', 'mean'),
    median_acreage=('lot_area_value', 'median')
).reset_index()

# Calculate absorption rate
zip_code_stats['absorption'] = zip_code_stats['properties_sold_30_days'] / zip_code_stats['properties_for_sale']

# Output the results to a CSV file
zip_code_stats.to_csv('/Users/work/Desktop/zip_code_stats.csv', index=False)
